In [1]:
import torch
import torch.optim as optim
import numpy as np
import random
from collections import defaultdict
import matplotlib.pyplot as plt
import math
import seaborn as sns

In [4]:
# create list of frequencies

# class Env1(object):
#     def __init__(self, T:int, omega:float, c:float, A:float):
#         self.T = T
#         self.omega = omega
#         self.c = c
#         self.eps = 10**-5
#         self.t = 0
        
class Options:
    """Class containing all parameters"""
    def __init__(self):
        # Env params
        self.T = 10
        self.omega = []
        self.c = 1
        self.eps = 10**-5
        self.A = 2
        self.t = 0
        self.lr = 1e-3
        self.timesteps = 20
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
opt = Options()

def f(x:torch.Tensor, t:int, omega:float):
    return alpha(t,omega) * (x.pow(2))
    
def alpha(t:int, omega:torch.Tensor):
    if t <= opt.T:
        return opt.c
    return (opt.A/2) * (1 + torch.sin(omega*t) + opt.eps)

In [18]:
adam_deltas = defaultdict(list)
sgd_deltas = defaultdict(list)
omegas = [torch.Tensor([2])]

for omega in omegas:
    x_adam = torch.randn(1)
    x_sgd = x_adam.clone()
    x_adam.requires_grad_(True)
    x_sgd.requires_grad_(True)
    
    adam_opt = optim.Adam([x_adam], lr=opt.lr)
    sgd_opt = optim.SGD([x_sgd], lr=opt.lr)
    
    for t in range(opt.timesteps):
        # Perform Adam and SGD steps
        y_adam = f(x_adam, t, omega)
        adam_opt.zero_grad()
        y_adam.backward()
        adam_opt.step()
        
        y_sgd = f(x_sgd, t, omega)
        sgd_opt.zero_grad()
        y_sgd.backward()
        sgd_opt.step()
        
        # Append to results
        adam_deltas[omega].append(x_adam.detach().item())
        sgd_deltas[omega].append(x_sgd.detach().item())

In [19]:
adam_deltas

defaultdict(list,
            {tensor([2.]): [1.7721784114837646,
              1.7711784839630127,
              1.7701785564422607,
              1.7691786289215088,
              1.7681787014007568,
              1.7671788930892944,
              1.766179084777832,
              1.7651793956756592,
              1.7641798257827759,
              1.7631802558898926,
              1.7621808052062988,
              1.7611819505691528,
              1.7602686882019043,
              1.7593140602111816,
              1.758341908454895,
              1.7574594020843506,
              1.7565360069274902,
              1.7555803060531616,
              1.7547109127044678,
              1.7538130283355713]})

In [14]:
sgd_deltas

defaultdict(list,
            {tensor([2.]): [-0.4028593599796295,
              -0.4020536541938782,
              -0.40124955773353577,
              -0.4004470705986023,
              -0.39964616298675537,
              -0.3988468647003174,
              -0.39804917573928833,
              -0.3972530663013458,
              -0.39645856618881226,
              -0.39566564559936523,
              -0.39487430453300476,
              -0.39409154653549194,
              -0.39401713013648987,
              -0.39262816309928894,
              -0.3916301727294922,
              -0.39162078499794006,
              -0.3904056251049042,
              -0.3892116844654083,
              -0.38920527696609497,
              -0.3881961703300476]})